In [1]:
# pip install sktime

In [2]:
import pandas as pd
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import numpy as np
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformations.series.detrend import Deseasonalizer
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sklearn.metrics import mean_squared_error
from sktime.forecasting.compose import make_reduction
from sklearn.exceptions import ConvergenceWarning
import requests
from sklearn.ensemble import (
    HistGradientBoostingRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.linear_model import ElasticNetCV
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    ExpandingWindowSplitter,
)
from sktime.forecasting.compose import MultiplexForecaster
from sklearn.neighbors import KNeighborsRegressor
from sktime.forecasting.ets import AutoETS
from sktime.transformations.series.boxcox import LogTransformer


import warnings

In [3]:
from sklearn.preprocessing import StandardScaler
from sktime.forecasting.compose import ForecastingPipeline
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.forecasting.arima import AutoARIMA


def initialize_arima_forecaster():
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            # ("seasonal", Deseasonalizer(sp=24, model="multiplicative")),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        (
                            "deseasonalizer_daily",
                            Deseasonalizer(sp=24, model="additive"),
                        ),
                        ("residual_forecaster", AutoARIMA(suppress_warnings=True)),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_elasticnet_forecaster():
    deseasonalizer_weekly = Deseasonalizer(sp=24 * 7, model="additive")
    # Create the TransformedTargetForecaster pipeline
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        # ("log_transformer", LogTransformer()),
                        (
                            "forecast",
                            make_reduction(
                                ElasticNetCV(n_jobs=-1),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_rf_forecaster():
    # Create the TransformedTargetForecaster pipeline
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        # ("log_transformer", LogTransformer()),
                        (
                            "forecast",
                            make_reduction(
                                RandomForestRegressor(n_estimators=100, n_jobs=-1),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_gb_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        # ("log_transformer", LogTransformer()),
                        (
                            "forecast",
                            make_reduction(
                                GradientBoostingRegressor(n_estimators=200),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_hist_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        # ("log_transformer", LogTransformer()),
                        (
                            "forecast",
                            make_reduction(
                                HistGradientBoostingRegressor(),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


def initialize_cat_forecaster():
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        # ("log_transformer", LogTransformer()),
                        (
                            "forecast",
                            make_reduction(
                                CatBoostRegressor(verbose=0, n_estimators=100),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_lgbm_forecaster():
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        # ("log_transformer", LogTransformer()),
                        (
                            "forecast",
                            make_reduction(
                                LGBMRegressor(),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_xgb_forecaster():
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        # ("log_transformer", LogTransformer()),
                        (
                            "forecast",
                            make_reduction(
                                XGBRegressor(objective="reg:squarederror"),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_ets_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")
    pipe = TransformedTargetForecaster(
        [
            # ("log_transformer", LogTransformer()),
            (
                "forecast",
                make_reduction(
                    AutoETS(auto=True, sp=24, n_jobs=-1),
                    window_length=24,
                    strategy="direct",
                ),
            ),
        ]
    )

In [4]:
arima_pipeline = initialize_arima_forecaster()
elasticnet_pipeline = initialize_elasticnet_forecaster()
rf_pipeline = initialize_rf_forecaster()
gb_pipeline = initialize_gb_forecaster()
hist_pipeline = initialize_hist_forecaster()
cat_pipeline = initialize_cat_forecaster()
lgbm_pipeline = initialize_lgbm_forecaster()
xgb_pipeline = initialize_xgb_forecaster()
ets_pipeline = initialize_ets_forecaster()


forecasting_models = {
    "arima_pipeline": arima_pipeline,
    "elasticnet_pipeline": elasticnet_pipeline,
    "rf_pipeline": rf_pipeline,
    "gb_pipeline": gb_pipeline,
    "hist_pipeline": hist_pipeline,
    # "ets_pipeline": ets_pipeline,
    "arima_pipeline": arima_pipeline,
    "cat_pipeline": cat_pipeline,
    "lgbm_pipeline": lgbm_pipeline,
    "xgb_pipeline": xgb_pipeline,
}

c:\Users\rkris\miniconda3\envs\slalomenv\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1480: UserWarning: The `scitype` of the given `estimator` cannot be inferred. Assuming "tabular-regressor" = scikit-learn regressor interface. If this warning is followed by an unexpected exception, please consider report as a bug on the sktime issue tracker.
  warn(


In [5]:
selected_features = [
    "ail",
    "gas_price",
    "gas_tng",
    "coal_tng",
    "wind_tng",
    "gas_avail",
    "wind_avail",
    "gas_reserve_margin",
    "coal_reserve_margin",
    "wind_reserve_margin",
    "other_reserve_margin",
    "gas_supply_mix",
    "coal_supply_mix",
    "wind_supply_mix",
    "other_supply_mix",
    "total_reserve_margin",
    "demand_supply_ratio",
    "fossil_fuel_ratio",
    "rolling_mean",
    "rolling_std",
    "rolling_min",
    "rolling_max",
    "rolling_median",
    "exp_moving_avg",
]

In [6]:
price_old_df = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/supply_load_price.csv",
    parse_dates=["Date (MST)"],
    index_col="Date (MST)",
)

window = 24
price_old_df = price_old_df.sort_values(by="Date (MST)")
price_old_df = price_old_df.asfreq("H")

price_old_df['rolling_mean'] = price_old_df['price'].rolling(window).mean().rolling(2).mean().shift(-window // 2)
price_old_df['rolling_std'] = price_old_df['price'].rolling(window).std().rolling(2).mean().shift(-window // 2)
price_old_df['rolling_min'] = price_old_df['price'].rolling(window).min().rolling(2).mean().shift(-window // 2)
price_old_df['rolling_max'] = price_old_df['price'].rolling(window).max().rolling(2).mean().shift(-window // 2)
price_old_df['rolling_median'] = price_old_df['price'].rolling(window).median().rolling(2).mean().shift(-window // 2)
price_old_df['exp_moving_avg'] = price_old_df['price'].ewm(span=24).mean()

In [7]:
price_old_df_filtered = price_old_df.loc["2023-01-01":"2023-01-31", selected_features + ["price"]]

y = price_old_df_filtered["price"]
X = price_old_df_filtered[selected_features]

# scale ratios to percentages
X["gas_supply_mix"] = X["gas_supply_mix"] * 100
X["coal_supply_mix"] = X["coal_supply_mix"] * 100
X["wind_supply_mix"] = X["wind_supply_mix"] * 100
X["other_supply_mix"] = X["other_supply_mix"] * 100
X['gas_reserve_margin'] = X['gas_reserve_margin'] * 100
X['coal_reserve_margin'] = X['coal_reserve_margin'] * 100
X['wind_reserve_margin'] = X['wind_reserve_margin'] * 100
X['other_reserve_margin'] = X['other_reserve_margin'] * 100
X['total_reserve_margin'] = X['total_reserve_margin'] * 100
X['demand_supply_ratio'] = X['demand_supply_ratio'] * 100
X['fossil_fuel_ratio'] = X['fossil_fuel_ratio'] * 100

C:\Users\rkris\AppData\Local\Temp\ipykernel_35384\3940189037.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["gas_supply_mix"] = X["gas_supply_mix"] * 100
C:\Users\rkris\AppData\Local\Temp\ipykernel_35384\3940189037.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["coal_supply_mix"] = X["coal_supply_mix"] * 100
C:\Users\rkris\AppData\Local\Temp\ipykernel_35384\3940189037.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [8]:
X.head()

,ail,gas_price,gas_tng,coal_tng,wind_tng,gas_avail,wind_avail,gas_reserve_margin,coal_reserve_margin,wind_reserve_margin,...,other_supply_mix,total_reserve_margin,demand_supply_ratio,fossil_fuel_ratio,rolling_mean,rolling_std,rolling_min,rolling_max,rolling_median,exp_moving_avg
Date (MST),,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,9824,4.12,7618.023188,817.176320,780.206753,8253.0,2619.0,7.693891,0.344351,70.209746,...,2.434244,21.349706,70.236648,76.677939,113.835833,62.836213,64.86,285.83,80.7350,111.610750
2023-01-01 01:00:00,9712,4.12,7688.218311,817.141767,732.207446,8252.0,2619.0,6.832061,0.348565,72.042480,...,2.400148,21.234191,69.440869,77.207847,113.087292,63.121695,64.86,285.83,80.6725,109.149090
2023-01-01 02:00:00,9623,4.12,7597.866907,816.632098,743.884060,8251.0,2619.0,7.915805,0.410720,71.596638,...,2.403357,21.750380,68.809439,76.892425,112.191667,63.487270,64.86,285.83,80.5900,106.867563
2023-01-01 03:00:00,9578,4.12,7537.204668,817.109768,759.293766,8247.0,2619.0,8.606709,0.352467,71.008256,...,2.428065,22.094972,68.507260,76.702072,109.710000,62.563158,64.86,285.83,80.5700,104.698958
2023-01-01 04:00:00,9599,4.12,7554.993344,816.817543,746.124896,8244.0,2619.0,8.357674,0.388104,71.511077,...,2.410568,21.871216,68.672199,76.659028,107.968542,62.674589,64.86,285.83,80.5700,102.685441


In [9]:
cv = ExpandingWindowSplitter(
    initial_window=int(len(X) * 0.94), step_length=1, fh=np.arange(1, 13)
)

n_splits = cv.get_n_splits(y)
print(f"Number of Folds = {n_splits}")

Number of Folds = 34


In [10]:
# from sktime.utils.plotting import plot_windows, get_windows
# train_windows, test_windows = get_windows(y, cv)
# plot_windows(y, train_windows, test_windows)

In [11]:
# from sktime.forecasting.model_evaluation import evaluate
# from sktime.performance_metrics.forecasting import MeanSquaredScaledError, MeanSquaredError

# list_models = ["arima_pipeline"]

# rmse_cv_results = []
# rmse_cv_std = []
# for i in list_models:
#     print(i)
#     results = evaluate(
#         forecaster=forecasting_models[i],
#         y=y,
#         X=X,
#         cv=cv,
#         strategy="refit",
#         return_data=True,
#         scoring=MeanSquaredError(square_root=True),
#         backend="loky",
#         error_score='raise'
#     )
    
#     rmse = results["test_MeanSquaredError"].mean()
#     rmse_std = results["test_MeanSquaredError"].std()
#     rmse_cv_results.append(rmse)
#     rmse_cv_std.append(rmse_std)

In [12]:
# rmse_cv_results_df = pd.DataFrame(
#     {"Model": list_models, "RMSE_CV": rmse_cv_results, "RMSE_CV_STD": rmse_cv_std}
# ).sort_values(by=["RMSE_CV"])
# rmse_cv_results_df

In [13]:
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredScaledError, MeanSquaredError

list_models = ["elasticnet_pipeline", "rf_pipeline", "gb_pipeline", "hist_pipeline", "cat_pipeline", "lgbm_pipeline", "xgb_pipeline"]

rmse_cv_results = []
rmse_cv_std = []
for i in list_models:
    print(i)
    results = evaluate(
        forecaster=forecasting_models[i],
        y=y,
        X=X,
        cv=cv,
        strategy="refit",
        return_data=True,
        scoring=MeanSquaredError(square_root=True),
        backend="loky",
        error_score='raise'
    )
    
    rmse = results["test_MeanSquaredError"].mean()
    rmse_std = results["test_MeanSquaredError"].std()
    rmse_cv_results.append(rmse)
    rmse_cv_std.append(rmse_std)

elasticnet_pipeline
rf_pipeline
gb_pipeline


: 

: 

In [ ]:
rmse_cv_results, rmse_cv_std

In [ ]:
rmse_cv_results_df = pd.DataFrame(
    {"Model": list_models, "RMSE_CV": rmse_cv_results, "RMSE_CV_STD": rmse_cv_std}
).sort_values(by=["RMSE_CV"])
rmse_cv_results_df

In [ ]:
price_old_df = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/supply_load_price.csv",
    parse_dates=["Date (MST)"],
    index_col="Date (MST)",
)

In [ ]:
train = price_old_df.loc["2023-01-01":"2023-01-31", selected_features+["price"]]
test = price_old_df.loc["2023-02-01":"2023-02-02", selected_features+["price"]]
train = train.sort_values(by="Date (MST)")
train = train.asfreq("H")
test = test.sort_values(by="Date (MST)")
test = test.asfreq("H")
X_train = train[selected_features]
y_train = train["price"]
X_test = test[selected_features]
y_test = test["price"]

In [ ]:
# fit and predict for all models
fh = np.arange(1, 13)
for i in list_models:
    print(i)
    forecasting_models[i].fit(y_train, X_train, fh=fh)
    y_pred = forecasting_models[i].predict(fh, X_train.tail(1))
    # get rmse between y_pred and y_test[:12]
    rmse = mean_squared_error(y_test[:12], y_pred, squared=False)
    print(rmse)